# Fetch Codebase and Install Environment

In [ ]:
import os
os.chdir('/content')
CODE_DIR = 'GenForce'
!git clone https://github.com/genforce/genforce.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
!pip install -r requirements.txt > installation_output.txt

# Define Utility Functions

In [ ]:
import os
import subprocess
import io
import IPython.display
import numpy as np
import PIL.Image

import torch

from models import MODEL_ZOO
from models import build_generator
from utils.visualizer import fuse_images


def postprocess(images):
  """Post-processes images from `torch.Tensor` to `numpy.ndarray`."""
  images = images.detach().cpu().numpy()
  images = (images + 1) * 255 / 2
  images = np.clip(images + 0.5, 0, 255).astype(np.uint8)
  images = images.transpose(0, 2, 3, 1)
  return images


def build(model_name):
  """Builds generator and load pre-trained weights."""
  model_config = MODEL_ZOO[model_name].copy()
  url = model_config.pop('url')  # URL to download model if needed.

  # Build generator.
  print(f'Building generator for model `{model_name}` ...')
  generator = build_generator(**model_config)
  print(f'Finish building generator.')

  # Load pre-trained weights.
  os.makedirs('checkpoints', exist_ok=True)
  checkpoint_path = os.path.join('checkpoints', model_name + '.pth')
  print(f'Loading checkpoint from `{checkpoint_path}` ...')
  if not os.path.exists(checkpoint_path):
    print(f'  Downloading checkpoint from `{url}` ...')
    subprocess.call(['wget', '--quiet', '-O', checkpoint_path, url])
    print(f'  Finish downloading checkpoint.')
  checkpoint = torch.load(checkpoint_path, map_location='cpu')
  if 'generator_smooth' in checkpoint:
    generator.load_state_dict(checkpoint['generator_smooth'])
  else:
    generator.load_state_dict(checkpoint['generator'])
  generator = generator.cuda()
  generator.eval()
  print(f'Finish loading checkpoint.')
  return generator


def synthesize(generator, num, synthesis_kwargs=None, batch_size=1, seed=0):
  """Synthesize images."""
  assert num > 0 and batch_size > 0
  synthesis_kwargs = synthesis_kwargs or dict()

  # Set random seed.
  np.random.seed(seed)
  torch.manual_seed(seed)

  # Sample and synthesize.
  outputs = []
  for idx in range(0, num, batch_size):
    batch = min(batch_size, num - idx)
    code = torch.randn(batch, generator.z_space_dim).cuda()
    with torch.no_grad():
      images = generator(code, **synthesis_kwargs)['image']
      images = postprocess(images)
    outputs.append(images)
  return np.concatenate(outputs, axis=0)


def imshow(images, viz_size=256, col=0, spacing=0):
  """Shows images in one figure."""
  fused_image = fuse_images(
    images,
    col=col,
    image_size=viz_size,
    row_spacing=spacing,
    col_spacing=spacing
  )
  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  PIL.Image.fromarray(fused_image).save(data, 'jpeg')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

# Select a Model

In [ ]:
#@title { display-mode: "form", run: "auto" }

model_name = "stylegan_diningroom256" #@param ['pggan_celebahq1024', 'pggan_bedroom256', 'pggan_livingroom256', 'pggan_diningroom256', 'pggan_kitchen256', 'pggan_churchoutdoor256', 'pggan_tower256', 'pggan_bridge256', 'pggan_restaurant256', 'pggan_classroom256', 'pggan_conferenceroom256', 'pggan_person256', 'pggan_cat256', 'pggan_dog256', 'pggan_bird256', 'pggan_horse256', 'pggan_sheep256', 'pggan_cow256', 'pggan_car256', 'pggan_bicycle256', 'pggan_motorbike256', 'pggan_bus256', 'pggan_train256', 'pggan_boat256', 'pggan_airplane256', 'pggan_bottle256', 'pggan_chair256', 'pggan_pottedplant256', 'pggan_tvmonitor256', 'pggan_diningtable256', 'pggan_sofa256', 'stylegan_ffhq1024', 'stylegan_celebahq1024', 'stylegan_bedroom256', 'stylegan_cat256', 'stylegan_car512', 'stylegan_celeba_partial256', 'stylegan_ffhq256', 'stylegan_ffhq512', 'stylegan_livingroom256', 'stylegan_diningroom256', 'stylegan_kitchen256', 'stylegan_apartment256', 'stylegan_church256', 'stylegan_tower256', 'stylegan_bridge256', 'stylegan_restaurant256', 'stylegan_classroom256', 'stylegan_conferenceroom256', 'stylegan_animeface512', 'stylegan_animeportrait512', 'stylegan_artface512', 'stylegan2_ffhq1024', 'stylegan2_church256', 'stylegan2_cat256', 'stylegan2_horse256', 'stylegan2_car512']

generator = build(model_name)

# Synthesize

In [ ]:
#@title { display-mode: "form", run: "auto" }

num_samples = 8 #@param {type:"slider", min:1, max:20, step:1}
noise_seed = 488 #@param {type:"slider", min:0, max:1000, step:1}
truncation = 1 #@param {type:"slider", min:0.0, max:1, step:0.02}
truncation_layers = 3 #@param {type:"slider", min:0, max:18, step:1}
randomize_noise = 'false' #@param ['true', 'false']

synthesis_kwargs = dict(trunc_psi=1 - truncation,
             trunc_layers=truncation_layers,
             randomize_noise=randomize_noise)
images = synthesize(generator, num_samples, synthesis_kwargs, seed=noise_seed)
imshow(images)